In [1]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

#from vit_pytorch.efficient import ViT
from vit_pytorch import SimpleViT, ViT

In [2]:
lr = 0.001 # learning_rate
batch_size = 100 # we will use mini-batch method
epochs = 10 # How much to train a model

In [3]:
device = torch.device('cuda')

torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)

In [4]:
train_dir = '../talData/images/train'
test_dir = '../talData/images/val'

train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))

print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

Train Data: 976
Test Data: 221


In [5]:
def get_labels(default_path, path):

    default_path = default_path
    file_name = path.split('/')[-1].split('.')[0] + '.json'

    tmp_path = default_path + file_name
    
    with open(tmp_path, 'r') as file:
        data = json.load(file)

    return data['image_info']['Movements']

labels = [get_labels('/kdh/talData/labelData/', i) for i in train_list]

In [6]:
    tmp = pd.DataFrame(np.unique(labels, return_counts = True)).T

    tmp.columns = ['talchumMovements', 'frameCnt']

    tmp

,talchumMovements,frameCnt
0,거울보기,56
1,곱사위,13
2,긴여다지,40
3,까치걸음,11
4,깨끼리,82
5,대기,1
6,멍석말이,50
7,몰아메기,29
8,반화장,27
9,배치기,18


In [6]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.1,
                                          stratify=labels,
                                          random_state=4326)

In [7]:
print(f"Train Data: {len(train_list)}")
print(f"Validation Data: {len(valid_list)}")
print(f"Test Data: {len(test_list)}")

Train Data: 878
Validation Data: 98
Test Data: 221


In [8]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((512, 512)),
        transforms.RandomResizedCrop(512),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize(513),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize(513),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
    ]
)

In [9]:
class talChumDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = get_labels('/kdh/talData/labelData/', img_path)
        label = 1 if label == 1 else 0

        return img_transformed, label


In [10]:
train_data = talChumDataset(train_list, transform=train_transforms)
valid_data = talChumDataset(valid_list, transform=test_transforms)
test_data = talChumDataset(test_list, transform=test_transforms)

In [11]:
train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )
val_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True)

In [13]:
print(len(train_data), len(train_loader))
print(len(valid_data), len(val_loader))

878 9
98 1


In [14]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn,self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
            )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        
        self.fc1 = nn.Linear(3136, 100)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(100,2)
        self.relu = nn.ReLU()
        
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [15]:
model = Cnn().to(device)
model.train()

Cnn(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3136, out_features=100, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=100, out_features=2, bias=True)
  

In [16]:
optimizer = optim.Adam(params = model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()

In [17]:
epochs = 10

for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    
    for data, label in train_loader:
        data = data.to(device)
        label = label.to(device)
        
        output = model(data)
        loss = criterion(output, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = ((output.argmax(dim=1) == label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
        
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))
    
    
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)
            
            val_output = model(data)
            val_loss = criterion(val_output,label)
            
            
            acc = ((val_output.argmax(dim=1) == label).float().mean())
            epoch_val_accuracy += acc/ len(val_loader)
            epoch_val_loss += val_loss/ len(val_loader)
            
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))

/home/dongha/miniconda3/envs/go/lib/python3.8/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch : 1, train accuracy : 0.42575499415397644, train loss : 1.055771827697754
Epoch : 1, val_accuracy : 0.36734694242477417, val_loss : 0.7046408653259277
Epoch : 2, train accuracy : 0.6131623387336731, train loss : 0.6640182733535767
Epoch : 2, val_accuracy : 0.6326530575752258, val_loss : 0.6606988906860352
Epoch : 3, train accuracy : 0.6288318634033203, train loss : 0.6329979300498962
Epoch : 3, val_accuracy : 0.6326530575752258, val_loss : 0.6038306951522827
Epoch : 4, train accuracy : 0.6263248324394226, train loss : 0.5910022854804993
Epoch : 4, val_accuracy : 0.6326530575752258, val_loss : 0.5565464496612549
Epoch : 5, train accuracy : 0.7192877531051636, train loss : 0.5396419167518616
Epoch : 5, val_accuracy : 0.7857142686843872, val_loss : 0.4590722322463989
Epoch : 6, train accuracy : 0.8294017314910889, train loss : 0.48778367042541504
Epoch : 6, val_accuracy : 0.7448979616165161, val_loss : 0.3914669156074524
Epoch : 7, train accuracy : 0.8513104915618896, train loss : 0

In [18]:
with torch.no_grad():
    test_accuracy = 0
    test_loss = 0

    for data, label in test_loader:
        data = data.to(device)
        label = label.to(device)

        test_out = model(data)
        test_loss = criterion(test_out, label)

        acc = (test_out.argmax(dim = 1) == label).float().mean()
        test_accuracy += acc / len(test_loader)
        test_loss += test_loss / len(test_loader)


print(f"test_loss : {test_loss:.4f} - test_acc : {test_accuracy:.4f}")

test_loss : 0.2460 - test_acc : 0.9408
